# Introduction

This notebook provides the base for suites of tests to run before, during and 
after maintenance windows.

**Usage:**

1. Create a spreadsheet that contains at least one workbook
    1. Put the name of one or more tests as column headers
    1. Put one or more device groups in each column
1. Enter the name of the spreadsheet and workbook when prompted below
1. Enter a filename to save the output to (it will be saved as a spreadsheet with a separate workbook for each test's output)
1. Run the "pre-check" cells
1. If you would like to run post-checks and compare the data, then run the "post-check" cells
    1. There is an option to import the results from the spreadsheet outputted in the pre-checks, so that post-checks can be run at a later date

**Methodology:**
1. Use Ansible-runner to execute Ansible playbooks, then return the output to Python for analysis
    1. If Ansible cannot be used for some reason, then an API or Netmiko can be used instead
1. I am coding this in Jupyter to allow for easy testing, but it can be ported to its own script(s) or a GUI later
1. I would like to coordinate with the authors of Get-Inventory to see if the functions can be re-used in that tool

**Functions:**

The following functions are preliminary; more will be added as planning continues.

1. **Cisco**
    1. **NXOS**
        1. Interface status
        1. Trunk / Port-channel status
        1. Spanning-tree (blocked ports, err-disabled ports, log entries)
        1. VPC status
        1. VLAN database
        1. ARP table (including in VRFs)
        1. CAM table
        1. Routes (including routes in VRFs)
        1. Routing neighbors
        1. VRFs
    1. **IOS / IOS-XE**
        1. Interface status
        1. Trunk / Port-channel status
        1. Spanning-tree (blocked ports, err-disabled ports, log entries)
        1. VLAN database
        1. ARP table (including in VRFs)
        1. CAM table
        1. Routes (including routes in VRFs)
        1. Routing neighbors
        1. VRFs
    1. **ASA (most functions are TBD)**
        1. Interface status
        1. Firewall rules
    1. **Meraki**
        1. TBD
1. **F5**
    1. **F5 LTM**
        1. Interface status
        1. Trunk status
        1. VLANs
        1. ARP table
        1. CAM table
        1. Routes
        1. Active VIPs
        1. Active Pools
        1. Active Pool Members
        1. Basic VIP testing (tentative plan is to use NMAP to check if expected ports are up)
        1. iRules (iRule syntax can change between software versions)
        1. iRule-specific testing (will vary by customer; for this customer, I will test HTTP redirects)
        1. **Future Functions That Can Be Added If Necessary:**
            1. Route domains
            1. Routing neighbors
            1. Spanning-tree
    1. **F5 Big-IQ**
        1. **TBD**
1. **Palo Alto**
    1. TBD

# Setup

## Import Modules and Set Various Parameters

This cell updates the MAC vendor database, which can take a couple of minutes.

In [ ]:
import ansible
import ansible_runner
import helpers as hp
import network_funcs
# import ipaddress
import nest_asyncio
import validators as vd
# import nmap3
import openpyxl
import os
import pandas as pd
import pathlib
import pykeepass
# import requests
import sys
# import yaml  # Remove after testing is done, because it is imported by the helpers library

from datetime import datetime as dt
from getpass import getpass
from pykeepass import PyKeePass
from IPython.display import display, HTML
from mac_vendor_lookup import MacLookup
from pprint import pprint

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.colheader_justify', 'center')
pd.set_option('display.precision', 3)

nest_asyncio.apply()
nb_path = str(pathlib.Path().resolve())

## Define Test Variables and Get User Credentials

Provide the necessary information in the following cell.

Provide the following data:
- Absolute path to the Keepass database containing tenant information
- Password to the Keepass Database
- Group Name

The group should contain the following entries (be sure the titles match):
- 'Tenant'
  - Contains the SSH username and password
- 'Net-Manage Path'
  - The path to the Net-Manage repository (put it in the 'URL' field)
- 'Ansible Private Data Directory'
  - The path to the Ansible Private Data Directory (in the 'URL' field)
- 'Tests File'
  - The path to the CSV file containing the tests to run

**TODO:** Add support for using different credentials for devices.

Here are example screenshots:

In [ ]:
kp_path = getpass('Enter the path to the Keepass database: ')
kp_path = os.path.expanduser(kp_path)
kp_pass = getpass('Enter your password to the Keepass database: ')
kp_group = getpass('Enter the group name that contains tenant data: ')

kp = PyKeePass(kp_path, kp_pass)
kp_group = kp.find_groups(name=kp_group, first=True)

uname = kp.find_entries(title='Tenant', group=kp_group)[0].username
pw = kp.find_entries(title='Tenant', group=kp_group)[0].password
nm = kp.find_entries(title='Net-Manage Path', group=kp_group)[0].url
tfile = kp.find_entries(title='Tests File', group=kp_group)[0].url
pdir = kp.find_entries(title='Private Data Dir', group=kp_group)[0].url
private_data_dir = os.path.expanduser(pdir)

df_tests, df_vars, nm_path, play_path, test_map = hp.map_tests_to_os(pdir,
                                                                     nm,
                                                                     tfile)

display(HTML('<h3>Tests to Run</h3>'))
display(df_tests)

display(HTML('<h3>Host Group Variables</h3>'))
display(df_vars)

# Run Tests

In [ ]:
import importlib
importlib.reload(network_funcs)
# Create a list of the columns in the df_tests dataframe. Each column will
# represent a host group.
cols = df_tests.columns.to_list()

# Iterate through the dictionary of tests. For each test, search the df_tests
# dataframe to see if the user requested that test. If they did, run the test
# and store the results in the 'test_map' dictionary.
for key, value in test_map.items():
    for c in cols:
        hostgroup = df_tests.loc[key][c]
        # If a test is requested, get the test name, ansible_network_os, and
        # playbook.
        if hostgroup != 'nan':
            test_name = key
            row = df_vars.loc[df_vars['host_group'] == hostgroup]
            ansible_os = row['ansible_network_os'][c]
            playbook = value.get(ansible_os)['playbook']
            result = nf.run_test(ansible_os,
                                 test_name,
                                 uname,
                                 pw,
                                 hostgroup,
                                 play_path,
                                 private_data_dir,
                                 nm_path)
            test_map[key][ansible_os]['result'] = result

In [ ]:
for key, value in test_map.items():
    if key == test_name:
        for k, v in value.items():
            result = value.get('cisco.nxos.nxos')['result']
            break
# display(result)
# df = result.loc[result['Device'] == 'CTS-CORE-SWT2'].copy()
display(result.info())
# display(result.loc[result['vendor'] == 'unknown'].info())
display(result)

In [ ]:
display(result)

In [ ]:
for key, value in test_map.items():
    if key == 'arp_table':
        for k, v in value.items():
            result = value.get('paloaltonetworks.panos')['result']
            break                
df = result.loc[result['Interface'] == 'vlan.123'].copy()
df.reset_index(inplace=True, drop=True)

import socket

reverse_dns = list()
for idx, row in df.iterrows():
    address = row['Address']
    try:
        rdns = socket.getnameinfo((address, 0), 0)[0]
    except Exception:
        rdns = 'unknown'
    reverse_dns.append(rdns)

df['Reverse DNS'] = reverse_dns
display(df)

In [ ]:
# Display NXOS Pre-checks

for key, value in test_map.items():
    for k, v in value.items():        
        if key == 'pre_post_checks':
            try:
                if pre_results:
                    pass
            except Exception:
                pre_results = value.get('cisco.nxos.nxos')['result']
            break

print('Available Dataframes:')
for key, value in pre_results.items():
    print(key)

display(pre_results['df_hostnames'])
display(pre_results['df_vpc_state'])
display(pre_results['df_vpc_status'])
display(len(pre_results['df_inf_status']))

df = pre_results['df_inf_status']
display(len(df.loc[df['Status'] == 'connected']))
display(df.loc[df['Status'] == 'connected'])

In [ ]:
del post_results


In [ ]:
# Display NXOS Post-checks

for key, value in test_map.items():
    for k, v in value.items():        
        if key == 'pre_post_checks':
            try:
                if post_results:
                    pass
            except Exception:
                post_results = value.get('cisco.nxos.nxos')['result']
            break

print('Available Dataframes:')
for key, value in post_results.items():
    print(key)

display(post_results['df_hostnames'])
display(post_results['df_vpc_state'])
display(post_results['df_vpc_status'])
display(len(post_results['df_inf_status']))

df = post_results['df_inf_status']
display(len(df.loc[df['Status'] == 'connected']))
display(df.loc[df['Status'] == 'connected'])

In [ ]:
for key, value in test_map.items():
    for k, v in value.items():
        df = v.get('result')
        if df is not None:
            display(v.get('result'))

## Create the Dictionary to Store Pre- and Post-check DataFrames

In [ ]:
checks = dict()
checks['f5s'] = dict()
checks['nxos'] = dict()

## Create a DataFrame Containing the Tests to Run

# Implementation

## Cisco Pre-checks

### Cisco NXOS Pre-checks

Runs the following tests:
* Get running-config diff
* Get err-disabled interfaces
* Get logs of err-disabled interfaces (to check for interfaces that auto-recovered)
* Get spanning-tree blocked ports
* Get VPC state
* Get VPC consistency
* Get VPC peer-keepalive

**ToDo:**

* Get ARP table
* Get CAM table (?)



    commands = {'diff_command': 'show running-config diff',
                'interface_status_command': 'show interface status err-disabled',
                'logging_command': 'show logging last 9999 | grep "err-disable\|BPDU"',
                'stp_blocked_ports_command': 'show spanning-tree blockedports',
                'vpc_brief_command': 'show vpc brief | grep Po',
                'vpc_consistency_command': 'show vpc consistency-parameters global | grep "Local suspended"',
                'vpc_keepalive_command': 'show vpc peer-keepalive | grep -v "ms\|msec"'}

In [ ]:
def cisco_nxos_run_checks(username,
                          password,
                          host_group,
                          play_path,
                          private_data_dir):
    '''
    Run the pre- and post-checks on a Cisco NXOS and adds the results to
    a dictionary. Also creates a list of devices in the host group.
    
    Args:
        username (str):         The username to login to devices
        password (str):         The password to login to devices
        host_group (str):       The inventory host group
        play_path (str):        The path to the playbooks directory
        private_data_dir (str): The path to the Ansible private data directory

    Returns:
        devices (list):    A list of devices in the host group
        check_data (dict): A dictionary containing the pre-checks
    '''
    # Create the base variables
    check_data = dict()
    
    # Define the commands to run
    commands = {'hostname_command': 'show hostname',
                'diff_command': 'show running-config diff',
                'interface_status_command': 'show interface status',
                'err_disabled_command': 'show interface status err-disabled',
                'logging_command': 'show logging last 9999 | grep "err-disable\|BPDU"',
                'stp_blocked_ports_command': 'show spanning-tree blockedports',
                'vpc_state_command': 'show vpc brief | begin "vPC domain id" | end "vPC Peer-link status"',
                'vpc_status_command': 'show vpc brief | begin "vPC status"',
                'vpc_consistency_command': 'show vpc consistency-parameters global | grep "Local suspended"',
                'vpc_keepalive_command': 'show vpc peer-keepalive | grep -v "ms\|msec"',
                'get_arp_table_command': 'show ip arp vrf all | begin "MAC Address"',
                'get_cam_table_command': 'show mac address-table  | begin "MAC Address"'}

    # Create the extra variables to pass to Ansible
    extravars = {'username': username,
                 'password': password,
                 'host_group': host_group,
                 'commands': cmd}

    # Execute the pre-checks
    runner = ansible_runner.run(private_data_dir='.',
                                playbook=f'{play_path}/cisco_nxos_pre_post_checks.yml',
                                extravars=extravars)
    
    # Parse the output; add the hosts to 'devices' and the command output to 'check_data'
    for event in runner.events:
        if event['event'] == 'runner_on_ok':
            event_data = event['event_data']
            device = event_data['remote_addr']
            
            # Create the dictionary keys for 'check_data'
            if not check_data.get(device):
                check_data[device] = dict()
            
            # Add the command output to 'check_data'
            cmd = event_data['res']['invocation']['module_args']['commands'][0]
            output = event_data['res']['stdout'][0].split('\n')
            check_data[device][cmd] = output

    return check_data

group = 'cts_core_7k'

pre_check_data = cisco_nxos_run_checks(username, password, group, play_path)

In [ ]:
def nxos_checks_dump(check_data):
    '''
    Formats the output of the 'cisco_nxos_pre_post_checks.yml' playbook and
    returns it as a dictionary.

    Args:
        check_data (dict): The output of the playbooks

    Returns:
        formatted_checks (dict): A nested dictionary, where the value for each
                                 key is the formatted output
    '''

    nxos_devices = list()
    
    check_hostnames = dict()
    check_hostnames['inventory_name'] = list()
    check_hostnames['hostname'] = list()

    check_inf_status = dict()
    check_inf_status['hostname'] = list()
    check_inf_status['Port'] = list()
    check_inf_status['Name'] = list()
    check_inf_status['Status'] = list()
    check_inf_status['Vlan'] = list()
    check_inf_status['Duplex'] = list()
    check_inf_status['Speed'] = list()
    check_inf_status['Type'] = list()

    check_vpc_status = dict()
    check_vpc_status['hostname'] = list()
    check_vpc_status['id'] = list()
    check_vpc_status['Port'] = list()
    check_vpc_status['Status'] = list()
    check_vpc_status['Consistency'] = list()
    check_vpc_status['Reason'] = list()
    check_vpc_status['Active vlans'] = list()

    check_vpc_state = dict()
    check_vpc_state['hostname'] = list()
    check_vpc_state['vPC domain id'] = list()
    check_vpc_state['Peer status'] = list()
    check_vpc_state['vPC keep-alive status'] = list()
    check_vpc_state['Configuration consistency status'] = list()
    check_vpc_state['Per-vlan consistency status'] = list()
    check_vpc_state['Type-2 consistency status'] = list()
    check_vpc_state['vPC role'] = list()
    check_vpc_state['Number of vPCs configured'] = list()
    check_vpc_state['Peer Gateway'] = list()
    check_vpc_state['Peer gateway excluded VLANs'] = list()
    check_vpc_state['Dual-active excluded VLANs'] = list()
    check_vpc_state['Graceful Consistency Check'] = list()
    check_vpc_state['Operational Layer3 Peer-router'] = list()
    check_vpc_state['Auto-recovery status'] = list()

    # Extract the ARP table
    check_arp_table = dict()
    check_arp_table['hostname'] = list()
    check_arp_table['Address'] = list()
    check_arp_table['Age'] = list()
    check_arp_table['MAC Address'] = list()
    check_arp_table['Interface'] = list()

    # Extract the CAM table
    check_cam_table = dict()
    check_cam_table['hostname'] = list()
    check_cam_table['Legend'] = list()
    check_cam_table['VLAN'] = list()
    check_cam_table['MAC Address'] = list()
    check_cam_table['Type'] = list()
    check_cam_table['age'] = list()
    check_cam_table['Secure'] = list()
    check_cam_table['NTFY'] = list()
    check_cam_table['Ports/SWID.SSID.LID'] = list()
    
    # Extract the spanning-tree blocked ports
    check_stp_blocked = dict()
    check_stp_blocked['hostname'] = list()
    check_stp_blocked['Name'] = list()
    check_stp_blocked['Blocked Interfaces List'] = list()
    
    # Extract the total number of blocked STP ports
    check_total_stp_blocked = dict()
    check_total_stp_blocked['hostname'] = list()
    check_total_stp_blocked['Total Blocked Ports'] = list()

    # Extract the total number of err-disabled ports
    check_err_disabled = dict()
    check_err_disabled['hostname'] = list()
    check_err_disabled['Port'] = list()
    check_err_disabled['Name'] = list()
    check_err_disabled['Status'] = list()
    check_err_disabled['Reason'] = list()
    
    for key, value in check_data.items():
        # Extract the hostnames
        nxos_devices.append(key)
        check_hostnames['inventory_name'].append(key)
        check_hostnames['hostname'].append(value['show hostname'][0])

        header = value['show interface status err-disabled'][1]
        pos_port = header.index('Port')
        pos_name = header.index('Name')
        pos_status = header.index('Status')
        pos_reason = header.index('Reason')
        for line in value['show interface status err-disabled'][3:]:
            try:
                port = line[pos_port:pos_name].strip()
                name = line[pos_name:pos_status].strip()
                status = line[pos_status:pos_reason].strip()
                reason = line[pos_reason:].strip()
                check_err_disabled['Port'].append(port)
                check_err_disabled['Name'].append(name)
                check_err_disabled['Status'].append(status)
                check_err_disabled['Reason'].append(reason)
                check_err_disabled['hostname'].append(key)
            except Exception:
                pass
        
        num_err_disabled_ports = len(check_err_disabled.get('Port'))
        
        for line in value['show spanning-tree blockedports'][2:]:
            line = line.split()
            if len(line) == 2:
                check_stp_blocked['hostname'].append(key)
                check_stp_blocked['Name'].append(line[0])
                check_stp_blocked['Blocked Interfaces List'].append(line[1])
            if '(segments)' in line:
                check_total_stp_blocked['hostname'].append(key)
                check_total_stp_blocked['Total Blocked Ports'].append(line[-1])
        
        for line in value['show ip arp vrf all | begin "MAC Address"'][1:]:
            check_arp_table['hostname'].append(key)
            check_arp_table['Address'].append(line.split()[0])
            check_arp_table['Age'].append(line.split()[1])
            check_arp_table['MAC Address'].append(line.split()[2])
            check_arp_table['Interface'].append(line.split()[3])        
        
        for line in value['show mac address-table  | begin "MAC Address"'][2:]:
            line = line.split()
            if len(line) == 8:
                check_cam_table['Legend'].append(line[0])
                check_cam_table['VLAN'].append(line[1])
                check_cam_table['MAC Address'].append(line[2])
                check_cam_table['Type'].append(line[3])
                check_cam_table['age'].append(line[4])
                check_cam_table['Secure'].append(line[5])
                check_cam_table['NTFY'].append(line[6])
                check_cam_table['Ports/SWID.SSID.LID'].append(line[7])
                check_cam_table['hostname'].append(key)

        
        
        # Extract the interface statuses
        header = value['show interface status'][1]
        pos = header.index('Status')
        for line in value['show interface status'][3:]:
            # Get the interface
            port = line.split()[0]

            # Get the interface name (this accounts for devices that change the
            # starting position of columns)
            line_prefix = line[:pos].split()
            name = ' '.join(line_prefix[1:])

            # Get the remaining parameters
            line_suffix = line[pos:].split()
            status = line_suffix[0]
            vlan = line_suffix[1]
            duplex = line_suffix[2]
            speed = line_suffix[3]
            try:
                port_type = line_suffix[4]
            except Exception:
                port_type = str()

            check_inf_status['hostname'].append(key)
            check_inf_status['Port'].append(port)
            check_inf_status['Name'].append(name)
            check_inf_status['Status'].append(status)
            check_inf_status['Vlan'].append(vlan)
            check_inf_status['Duplex'].append(duplex)
            check_inf_status['Speed'].append(speed)
            check_inf_status['Type'].append(port_type)

        # Extract the VPC state information
        vpc_state = value['show vpc brief | begin "vPC domain id" | end "vPC Peer-link status"'][:-2]
        check_vpc_state['hostname'].append(key)
        for line in vpc_state:
            line = line.split(':')
            line = [l.strip() for l in line]
            check_vpc_state[line[0]].append(line[-1])

        vpc_status = value['show vpc brief | begin "vPC status"'][4:]
        vpc_status = [l for l in vpc_status if len(l.split()) != 1]
        for line in vpc_status:
            line = line.split()
            if len(line) == 6 or len(line) == 8:
                id_ = line[0]
                port = line[1]
                status = line[2]
                if len(line) == 8:
                    consistency = 'Not Applicable'
                    reason = 'Consistency Check Not Performed'
                else:
                    consistency = line[3]
                    reason = line[4]
                active_vlans = line[-1]
                check_vpc_status['hostname'].append(key)
                check_vpc_status['id'].append(id_)
                check_vpc_status['Port'].append(port)
                check_vpc_status['Status'].append(status)
                check_vpc_status['Consistency'].append(consistency)
                check_vpc_status['Reason'].append(reason)
                check_vpc_status['Active vlans'].append(active_vlans)        

    if check_vpc_state.get('Peer Gateway'):
        vpc_in_use = True
    else:
        vpc_in_use = False
                
    formatted_checks = dict()
    formatted_checks['check_arp_table'] = check_arp_table
    formatted_checks['check_cam_table'] = check_cam_table
    formatted_checks['check_err_disabled'] = check_err_disabled
    formatted_checks['num_err_disabled_ports'] = num_err_disabled_ports
    formatted_checks['check_hostnames'] = check_hostnames
    formatted_checks['check_inf_status'] = check_inf_status
    formatted_checks['check_stp_blocked'] = check_stp_blocked
    formatted_checks['check_total_stp_blocked'] = check_total_stp_blocked
    formatted_checks['check_vpc_status'] = check_vpc_status
    formatted_checks['check_vpc_state'] = check_vpc_state
    formatted_checks['vpc_in_use'] = vpc_in_use
                
    return formatted_checks


pre_formatted_checks = nxos_checks_dump(pre_check_data)
pre_check_arp_table = pre_formatted_checks['check_arp_table']
pre_check_cam_table = pre_formatted_checks['check_cam_table']
pre_check_err_disabled = pre_formatted_checks['check_err_disabled']
pre_check_num_err_disabled_ports = pre_formatted_checks['num_err_disabled_ports']
pre_check_hostnames = pre_formatted_checks['check_hostnames']
pre_check_inf_status = pre_formatted_checks['check_inf_status']
pre_check_stp_blocked = pre_formatted_checks['check_stp_blocked']
pre_check_total_stp_blocked = pre_formatted_checks['check_total_stp_blocked']
pre_check_vpc_status = pre_formatted_checks['check_vpc_status']
pre_check_vpc_state = pre_formatted_checks['check_vpc_state']
vpc_in_use = pre_formatted_checks['vpc_in_use']

df_pre_arp_table = pd.DataFrame.from_dict(pre_check_arp_table)
del df_pre_arp_table['Age']
# display(df_pre_arp_table)

df_pre_hostnames = pd.DataFrame.from_dict(pre_check_hostnames)
checks['nxos']['hostnames'] = df_pre_hostnames
display(df_pre_hostnames)

if pre_check_num_err_disabled_ports:
    df_pre_err_disabled = pd.DataFrame.from_dict(pre_check_err_disabled)
    checks['nxos']['err_disabled_ports'] = df_pre_err_disabled
    display(df_pre_err_disabled)

df_pre_check_stp_blocked = pd.DataFrame.from_dict(pre_check_stp_blocked)
checks['nxos']['stp_blocked_ports'] = df_pre_check_stp_blocked
# display(df_pre_check_stp_blocked)

df_pre_check_total_stp_blocked = pd.DataFrame.from_dict(pre_check_total_stp_blocked)
checks['nxos']['total_stp_blocked_ports'] = df_pre_check_total_stp_blocked
# display(df_pre_check_total_stp_blocked)

df_pre_inf_status = pd.DataFrame.from_dict(pre_check_inf_status)
checks['nxos']['interface_statuses'] = df_pre_inf_status
# display(df_pre_inf_status)

df_arp_table = pd.DataFrame.from_dict(pre_check_arp_table)
checks['nxos']['arp_table'] = df_arp_table

df_pre_cam_table = pd.DataFrame.from_dict(pre_check_cam_table)
checks['nxos']['cam_table'] = df_pre_cam_table
# display(df_pre_cam_table)

if vpc_in_use:
    df_pre_vpc_state = pd.DataFrame.from_dict(pre_check_vpc_state)
    checks['nxos']['vpc_state'] = df_pre_vpc_state
    display(df_pre_vpc_state)

    df_pre_vpc_status = pd.DataFrame.from_dict(pre_check_vpc_status)
    checks['nxos']['vpc_status'] = df_pre_vpc_status
    display(df_pre_vpc_status)

## F5 Pre-checks

### Implementation: Run Pre-checks

In [ ]:
group = 'f5_ltm_cts'

In [ ]:
cmds = {'show_hostname': 'list sys global-settings hostname | grep hostname',
        'sync_status': 'show cm sync-status | grep Status | grep -v CM',
        'sys_version': 'show sys version | grep Version | grep -v Sys',
        'total_vips': 'list ltm virtual /*/* | grep -c "ltm virtual"',
        'vip_availability': 'show ltm virtual /*/* | grep "Ltm::\|Availability"',
        'available_vips': 'show ltm virtual /*/* | grep Availability | grep -c available',
        'vip_destinations': 'show ltm virtual /*/* | grep "Virtual Server\|Availability\|Destination"',
        'active_pool_members': 'show ltm pool /*/* | grep "Current Active Members"',
        'pool_member_availability': 'show ltm pool /*/* members detail | grep "Ltm::Pool\|Availability"',
        'get_arp_table': 'show net arp | grep -v "\-\-\-\|Net::Arp\|HWaddress"',  # TODO: Does this command display ARP entries for all partitions?
        'get_cam_table': 'show sys mac-address | grep "mac-address"',
        'show_net_interface': 'show net interface | grep -v "\-\-\-\|Net::Interface\|Bits\|Out"',
        'list_net_interface': 'list net interface',
        # 'get_irules': 'list ltm rule /*/*'
       }

extravars = {'host_group': group,
             'user': username,
             'password': password,
             'validate_certs': 'no'}

for cmd in cmds:
    extravars[cmd] = cmds[cmd]
    
devices = list()

results = dict()

runner = ansible_runner.run(private_data_dir='.',
                            playbook=f'{play_path}/f5_pre_post_checks.yml',
                            extravars=extravars)

print(f'{runner.status}: {runner.rc}')

results = dict()

for event in runner.events:
    if event['event'] == 'runner_on_ok':
        # Parse the event to extract the data
        event_data = event['event_data']
        host = event_data['host']
        cmd = event_data['res']['invocation']['module_args']['commands'].pop()
        result = event_data['res']['stdout'][0]

        # If a nested dictionary for the host does not yet exist, then create it
        if not results.get(host):
            results[host] = dict()

        # Map the command to the corresponding key in the 'cmds' dictionary and add
        # it to the result.
        for key, value in cmds.items():
            if value == cmd:
                results[host][key] = result
                break

        # Add the host to the 'devices' list
        if host not in devices:
            devices.append(host)


In [ ]:
# Create a dictionary to store the pre-check data
pre_checks = dict()
pre_checks['device'] = list()
for cmd in cmds:
    pre_checks[cmd] = list()

# Remove dictionary keys that we do not need for 'df_pre'
excluded_cmds = ['vip_availability',
                 'pool_member_availability',
                 'get_arp_table',
                 'get_cam_table',
                 # 'show_net_interface',
                 'vip_destinations',
                 'list_net_interface'
                 ]
# for cmd in excluded_cmds:
#     del pre_checks[cmd]

# Parse the 'results' dictionary and add the applicable output to 'df_pre'
for device in devices:
    for key, value in results[device].items():
        if key == 'show_hostname':
            # pprint(value)
            hostname = value.split()[-1]
        if key == 'sync_status':
            sync_status = value.split('Status')[-1].strip()
        if key == 'sys_version':
            sys_version = value.split()[-1]
        if key == 'total_vips':
            total_vips = value
        if key == 'available_vips':
            available_vips = value

        if key == 'vip_destinations':
            vip_destinations = dict()
            vip_destinations['hostname'] = list()
            vip_destinations['name'] = list()
            vip_destinations['availability'] = list()
            vip_destinations['destination'] = list()
            vip_destinations['port'] = list()
            for line in value.split('\n'):
                if line.split()[0] == 'Ltm::Virtual':
                    vip_destinations['hostname'].append(device)
                    name = line.split()[-1]
                    vip_destinations['name'].append(name)
                if line.split()[0] == 'Availability':
                    availability = line.split()[-1]
                    vip_destinations['availability'].append(availability)
                if line.split()[0] == 'Destination':
                    destination = line.split()[-1].split(':')[0]
                    vip_destinations['destination'].append(destination)
                    port = line.split()[-1].split(':')[-1]
                    vip_destinations['port'].append(port)
            df_vip_destinations = pd.DataFrame.from_dict(vip_destinations)
            
        if key == 'active_pool_members':
            num_members = 0
            for line in value.split('\n'):
                num_members += int(line.split()[-1])
            active_pool_members = str(num_members)
        if key == 'show_net_interface':
            num_up_interfaces = 0
            for line in value.split('\n'):
                if line.split()[1] == 'up':
                    num_up_interfaces += 1
    
    # Add the results to the 'pre_checks' dictionary
    pre_checks['device'].append(device)
    pre_checks['show_hostname'].append(hostname)
    pre_checks['sync_status'].append(sync_status)
    pre_checks['sys_version'].append(sys_version)
    pre_checks['total_vips'].append(total_vips)
    pre_checks['available_vips'].append(available_vips)
    pre_checks['active_pool_members'].append(active_pool_members)
    pre_checks['show_net_interface'].append(num_up_interfaces)

# Try to delete the 'df_pre' dataframe. This is to avoid corruption if it is recreated
try:
    del df_pre
except Exception:
    pass

# Remove from 'pre_checks' any keys that are in excluded_cmds
to_delete = list()
for key, value in pre_checks.items():
    if key in excluded_cmds:
        to_delete.append(key)
for cmd in to_delete:
    del pre_checks[cmd]

checks['f5s']['summary'] = pd.DataFrame.from_dict(pre_checks)

display(checks['f5s']['summary'])

### Create Additional Dataframes

In [ ]:
# Create a dataframe for individual pool member states
pre_member_states = dict()
pre_member_states['device'] = list()
pre_member_states['pool_name'] = list()
pre_member_states['pool_member'] = list()
pre_member_states['pool_member_state'] = list()

# Iterate through 'results', adding pool members states to 'pre_member_states'
for device in devices:
    member_states = results[device]['pool_member_availability'].split('\n')
    for line in member_states:
        if 'Ltm::Pool:' in line:
            pool_name = line.split('Ltm::Pool: ')[-1]
            try:
                pos = member_states.index(line)+1
                while 'Ltm::Pool:' not in member_states[pos]:
                    if '| Ltm::Pool Member:' in member_states[pos]:
                        pool_member = member_states[pos].split('Ltm::Pool Member: ')[-1]
                        pool_member_state = member_states[pos+1].split(':')[-1].strip()

                        pre_member_states['device'].append(device)
                        pre_member_states['pool_name'].append(pool_name)
                        pre_member_states['pool_member'].append(pool_member)
                        pre_member_states['pool_member_state'].append(pool_member_state)
                    pos += 1
            except Exception as e:
                if str(e) == 'list index out of range':
                    break
                else:
                    print(str(e))
                    
# Try to delete 'df_pre_member_state'
try:
    del df_pre_member_state
except Exception:
    pass

# Create 'df_pre_member_state'
df_pre_member_state = pd.DataFrame.from_dict(pre_member_states)
checks['f5s']['pool_member_states'] = df_pre_member_state

display(df_pre_member_state.head(10))

# Create a dataframe for individual VIP states
pre_vip_states = dict()
pre_vip_states['device'] = list()
pre_vip_states['vip_name'] = list()
pre_vip_states['vip_state'] = list()

# Iterate through 'results', adding VIP states to 'pre_vip_states'
for device in devices:
    vip_availability = results[device]['vip_availability'].split('\n')
    for line in vip_availability:
        if 'Ltm::Virtual Server:' in line:
            # Get the position of the line in the list
            pos = vip_availability.index(line)

            # Use the position to get the VIP availability
            vip_state = vip_availability[pos+1].split()[-1]
            
            # Extract the VIP name from the line
            vip_name = line.split('Ltm::Virtual Server: ')[-1]

            # Add the data to 'pre_vip_states'
            pre_vip_states['device'].append(device)
            pre_vip_states['vip_name'].append(vip_name)
            pre_vip_states['vip_state'].append(vip_state)
            
# Try to delete 'df_pre_vip_state'
try:
    del df_pre_vip_state
except Exception:
    pass

# Create 'df_pre_vip_state'
df_pre_vip_state = pd.DataFrame.from_dict(pre_vip_states)
checks['f5s']['vip_states'] = df_pre_vip_state
display(df_pre_vip_state.head(10))

# Create a dataframe for the ARP table
# Define the dataframe columns
cols = ['name',
        'address',
        'hw_address',
        'vlan',
        'expire_in_sec',
        'status']



# Create a dictionary to store the ARP table data
pre_arp = dict()
pre_arp['device'] = list()
for c in cols:
    pre_arp[c] = list()

# Iterate through 'results', adding ARP table data to 'pre_arp'
for device in devices:
    arp_table = results[device]['get_arp_table'].split('\n')
    for line in arp_table:
        pre_arp['device'].append(device)
        pre_arp['name'].append(line.split()[0])
        pre_arp['address'].append(line.split()[1])
        pre_arp['hw_address'].append(line.split()[2])
        pre_arp['vlan'].append(line.split()[3])
        pre_arp['expire_in_sec'].append(line.split()[4])
        pre_arp['status'].append(line.split()[5])
            
# Try to delete 'pre_arp'
try:
    del df_pre_arp
except Exception:
    pass

# Create 'df_pre_arp'
df_pre_arp = pd.DataFrame.from_dict(pre_arp)
checks['f5s']['arp_table'] = df_pre_arp
display(df_pre_arp.head(10))




# Create a dataframe for the CAM table
cols = ['device',
        'entry',
        'component',
        'object_id',
        'property']

# Create a dictionary to store the CAM table data
pre_cam = dict()
pre_cam['device'] = list()
for c in cols:
    pre_cam[c] = list()

# Iterate through 'results', adding ARP table data to 'pre_arp'
for device in devices:
    cam_table = results[device]['get_cam_table'].split('\n')
    for line in cam_table:
        pre_cam['device'].append(device)
        pre_cam['entry'].append(line.split()[0])
        pre_cam['component'].append(line.split()[1])
        pre_cam['object_id'].append(line.split()[2])
        pre_cam['property'].append(line.split()[3])
            
# Try to delete 'df_pre_cam'
try:
    del df_pre_cam
except Exception:
    pass

# Create 'df_pre_cam'
df_pre_cam = pd.DataFrame.from_dict(pre_cam)
checks['f5s']['cam_table'] = df_pre_cam
display(df_pre_cam.head(10))

### Use NMAP To Scan Available VIP Ports

In [ ]:
destinations_all = df_vip_destinations['destination'].to_list()
destinations_any = df_vip_destinations.loc[df_vip_destinations['port'] == 'any']['destination'].to_list()

ports = df_vip_destinations['port'].to_list()
ports = [p for p in ports if p != 'any']
ports = list(set(ports))

top_port = 0
for p in ports:
    if int(p) > top_port:
        top_port = int(p)

len(df_vip_destinations), len(destinations_all), len(destinations_any), len(ports), top_port

checks['f5s']['vip_destinations'] = df_vip_destinations

display(checks['f5s']['vip_destinations'].head(10))

In [ ]:
nmap = nmap3.Nmap()
print('Scanning destinations that have defined ports')
results_unique = nmap.scan_top_ports(' '.join(destinations_all),
                                     args=f'-np {",".join(ports)}',
                                     default=top_port)

print('Doing a "top ports" scan of ports with a destination of "any"')
results_any = nmap.scan_top_ports(' '.join(destinations_any),
                                  args='-n')

def remove_invalid_keys(result):
    invalid_keys = list()
    for key in result:
        try:
            if ipaddress.ip_address(key):
                pass
        except Exception:
            invalid_keys.append(key)
    for key in invalid_keys:
        del result[key]
    return result

results_unique = remove_invalid_keys(results_unique)
results_any = remove_invalid_keys(results_any)
      
len(df_vip_destinations), len(results_unique), len(results_any)

In [ ]:
open_ports = list()
closed_ports = list()

scan_dict = dict()
for key, value in results_unique.items():
    scan_dict[key] = dict()
    scan_dict[key]['open'] = list()
    scan_dict[key]['closed'] = list()
    
    for item in value['ports']:
        if item.get('state'):
            port = item.get('portid')
            state = item.get('state')
            if state == 'open':
                scan_dict[key]['open'].append(port)
            else:
                scan_dict[key]['closed'].append(port)

for key, value in results_any.items():
    if not scan_dict.get(key):
        scan_dict[key] = dict()
        scan_dict[key]['open'] = list()
        scan_dict[key]['closed'] = list()
    
    for item in value['ports']:
        if item.get('state'):
            state = item.get('state')
            port = item.get('portid')
            
            if state == 'open':
                scan_dict[key]['open'].append(port)
            else:
                scan_dict[key]['closed'].append(port)            
        
for idx, row in df_vip_destinations.iterrows():
    destination = row['destination']
    if scan_dict.get(destination):
        p_open = ' '.join(scan_dict[destination].get('open'))
        p_closed = ' '.join(scan_dict[destination].get('closed'))
        
    else:
        p_open = 'unknown'
        p_closed = 'unknown'
        
    open_ports.append(p_open)
    closed_ports.append(p_closed)

df_vip_destinations['open ports'] = open_ports
df_vip_destinations['closed ports'] = closed_ports
    
len(df_vip_destinations), len(open_ports), len(closed_ports)

In [ ]:
df_http = df_vip_destinations.loc[df_vip_destinations['availability'] == 'available'].copy()

df_available_http_vips = df_http.loc[(df_http['port'] == '80') |
                                     (df_http['port'] == '443') | 
                                     (df_http['port'] == '8080')]

checks['f5s']['available_http_vips'] = df_available_http_vips

display(df_available_http_vips)

### Check if Web Site Responds

In [ ]:
http_responses = dict()
http_responses['name'] = list()
http_responses['destination'] = list()
http_responses['size'] = list()
http_responses['error'] = list()

for idx, row in df_available_http_vips.iterrows():
    destination = row['destination']
    website = f'http://{destination}'
    try:
        r = requests.get(website, timeout=15)
        http_responses['name'].append(row['name'])
        http_responses['destination'].append(destination)
        http_responses['size'].append(len(r.content))
        error = str(0)
        http_responses['error'].append(error)
    except Exception as e:
        http_responses['name'].append(row['name'])
        http_responses['destination'].append(destination)
        http_responses['size'].append(0)
        error = str(e)
        http_responses['error'].append(error)

df_http_responses = pd.DataFrame.from_dict(http_responses)

checks['f5s']['http_responses'] = df_http_responses

display(df_http_responses)

# Save the Checks to a Spreadsheet

In [ ]:
filename = set_filename(filepath)
filename

with pd.ExcelWriter(filename) as writer:
    for key, value in checks.items():
        for k, v in value.items():
            sheet_name = f'{key}_{k}'
            v.to_excel(writer, sheet_name=sheet_name, index=False)
            
print(f'File saved to {filename}')

## Compare Spreadsheets

In [ ]:
comp_files = input('Enter the names (comma-delimited) of two spreadsheets to compare: ')
comp_files = [f.strip() for f in comp_files.split(',')]
comp_files

In [ ]:
try:
    del dict_1
except Exception:
    pass

try:
    del dict_2
except Exception:
    pass

dict_1 = pd.read_excel(comp_files[0], sheet_name=None)
dict_2 = pd.read_excel(comp_files[1], sheet_name=None)

In [ ]:
status_dict = dict()
status_dict['f5s_pool_member_states'] = list()
status_dict['f5s_vip_states'] = list()
status_dict['f5s_arp_table'] = list()
status_dict['f5s_vip_destinations'] = list()
status_dict['f5s_available_http_vips'] = list()

for key, value in dict_1.items():
    try:
        del df_1
    except Exception:
        pass
    try:
        del df_2
    except Exception:
        pass

    df_1 = value.copy()
    df_2 = dict_2[key].copy()
    
    if key == 'f5s_summary':
        display(df_1)
        display(df_2)
    
    if key == 'f5s_pool_member_states':
        status_dict[key].append(len(df_1.loc[df_1['pool_member_state'] == 'available']))
        status_dict[key].append(len(df_2.loc[df_2['pool_member_state'] == 'available']))                     
        # print(key, len(df_1.loc[df_1['pool_member_state'] == 'available']), len(df_2.loc[df_2['pool_member_state'] == 'available']))
        
    if key == 'f5s_vip_states':
        status_dict[key].append(len(df_1.loc[df_1['vip_state'] == 'available']))
        status_dict[key].append(len(df_2.loc[df_2['vip_state'] == 'available']))
        # print(key, len(df_1.loc[df_1['vip_state'] == 'available']), len(df_2.loc[df_2['vip_state'] == 'available']))
        
    if key == 'f5s_arp_table':
        status_dict[key].append(len(df_1.loc[df_1['status'] == 'resolved']))
        status_dict[key].append(len(df_2.loc[df_2['status'] == 'resolved']))
        # print(key, len(df_1.loc[df_1['status'] == 'resolved']), len(df_2.loc[df_2['status'] == 'resolved']))
        
    if key == 'f5s_vip_destinations':
        status_dict[key].append(len(df_1.loc[df_1['availability'] == 'available']))
        status_dict[key].append(len(df_2.loc[df_2['availability'] == 'available']))
        # print(key, len(df_1.loc[df_1['availability'] == 'available']), len(df_2.loc[df_2['availability'] == 'available']))

    if key == 'f5s_available_http_vips':
        status_dict[key].append(len(df_1.loc[df_1['availability'] == 'available']))
        status_dict[key].append(len(df_2.loc[df_2['availability'] == 'available']))

df_status = pd.DataFrame.from_dict(status_dict).T
# df_status.rename(columns = {'0': comp_files[0], '1': comp_files[1]}, inplace=True)
df_status.columns = [comp_files[0], comp_files[1]]

display(df_status)

In [ ]:
for key, value in dict_1.items():
    print(key)

In [ ]:
for key, value in dict_1.items():
    print(key)
    print('\n')
    print(value.info())
    print(dict_2[key].info())
    display(value.head(10))
    display(dict_2[key].head(10))
    print('\n')
    print('\n')